In [17]:
# ─── Celda 1: Definición de esquemas y funciones auxiliares ─────────────────────────────
from pyspark.sql import functions as F
from pyspark.sql.types import *
import uuid
import pandas as pd
from notebookutils import mssparkutils
import tempfile
import os

# 1. Definición de esquemas

sample_schema = StructType([
    StructField("uuid", StringType(), False),
    StructField("SampleID", StringType(), True),       # Será el valor de CODE
    StructField("LocationID", StringType(), True),      # Siempre null por ahora
    StructField("SampleType", StringType(), True),      # “Sample Type” en el Excel
    StructField("MaterialType", StringType(), True),    # “Material group” en el Excel
    StructField("State", StringType(), True),           # Siempre null
    StructField("DepthFrom_m", DoubleType(), True),
    StructField("DepthTo_m", DoubleType(), True),
    StructField("MiddleDepth_m", DoubleType(), True),
    StructField("Laboratory", StringType(), True),      # Siempre null
    StructField("Comment", StringType(), True),         # Siempre null
    StructField("createdAt", TimestampType(), False)
])

labtest_schema = StructType([
    StructField("uuid", StringType(), False),
    StructField("LabTestID", StringType(), False),      # Valor de “Test”
    StructField("SampleID", StringType(), False),       # Será el uuid generado para el sample
    StructField("TestType", StringType(), True),        # Valor de “CD/CU”
    StructField("TestDate", DateType(), True),          # Siempre null
    StructField("ReceivedDate", DateType(), True),      # Siempre null
    StructField("Comment", StringType(), True),         # Siempre null
    StructField("createdAt", TimestampType(), False)
])

triaxial_schema = StructType([
    StructField("uuid", StringType(), False),
    StructField("LabTestID", StringType(), False),
    StructField("TriaxialType", StringType(), True),
    StructField("Direction", StringType(), True),
    StructField("ConsolidationType", StringType(), True),
    StructField("AxialStrain_Percent", DoubleType(), True),
    StructField("VolumetricStrain_Percent", DoubleType(), True),
    StructField("MajorPrincipalEffectiveStress_kPa", DoubleType(), True),
    StructField("MinorPrincipalEffectiveStress_kPa", DoubleType(), True),
    StructField("ExcessPorePressure_kPa", DoubleType(), True),
    StructField("VoidRatio", DoubleType(), True),
    StructField("createdAt", TimestampType(), False)
])

# 2. Función para generar UUID determinista
def generate_uuid(specimen):
    namespace_uuid = uuid.UUID('6ba7b810-9dad-11d1-80b4-00c04fd430c8')
    return str(uuid.uuid5(namespace_uuid, str(specimen)))

# Registrar UDF para generar UUID en Spark
spark.udf.register("generate_uuid_udf", generate_uuid, StringType())

# 3. Función para cálculo de profundidades (struct con DepthFrom_m, DepthTo_m, MiddleDepth_m)
@F.udf(returnType=StructType([
    StructField("DepthFrom_m", DoubleType()),
    StructField("DepthTo_m", DoubleType()),
    StructField("MiddleDepth_m", DoubleType())
]))
def calculate_depths(depth_str):
    if depth_str is None:
        return (None, None, None)
    try:
        depth_str = str(depth_str).lower().replace("m", "").strip()
        if "-" in depth_str:
            parts = depth_str.split("-")
            from_depth = float(parts[0].strip())
            to_depth = float(parts[1].strip())
            return (from_depth, to_depth, (from_depth + to_depth) / 2)
        else:
            depth = float(depth_str)
            return (depth, depth, depth)
    except:
        return (None, None, None)

# 4. Función para cargar archivos Excel (usa mssparkutils y pandas)
def load_excel(file_path):
    """Carga un archivo Excel desde Files usando mssparkutils y retorna un Spark DataFrame."""
    temp_dir = tempfile.gettempdir()
    local_file = os.path.join(temp_dir, os.path.basename(file_path))
    mssparkutils.fs.cp(file_path, f"file:{local_file}", True)
    pdf = pd.read_excel(local_file)
    return spark.createDataFrame(pdf)

# 5. Función para procesar datos triaxiales hoja por hoja (igual que antes)
def process_triaxial_sheet(row, excel_data_path):
    try:
        sheet_name = row["Sheetname"]
        first_row = int(row["Fisrt row"])   # p.ej. 2
        last_row = int(row["Last row"])     # p.ej. 145

        temp_dir = tempfile.gettempdir()
        local_file = os.path.join(temp_dir, os.path.basename(excel_data_path))
        mssparkutils.fs.cp(excel_data_path, f"file:{local_file}", True)

        # Leer la hoja con pandas: skiprows=first_row-1, header=[0,1]
        df_sheet = pd.read_excel(
            local_file,
            sheet_name=sheet_name,
            skiprows=first_row - 1,
            header=[0, 1]
        )
        if df_sheet.empty:
            print(f"⚠️ Advertencia: Hoja vacía - Test: {row['Test']}, Hoja: {sheet_name}")
            return None

        # Aplanar MultiIndex de columnas concatenando ambos niveles
        df_sheet.columns = [
            f"{str(col[0]).strip()}_{str(col[1]).strip()}"
            for col in df_sheet.columns.to_list()
        ]

        # Calcular cuántas filas de datos reales hay: last_row - first_row - 1
        num_datos = last_row - first_row - 1
        df_sheet = df_sheet.iloc[:num_datos]

        # Renombrar columnas de pandas al esquema triaxial_schema
        column_mapping = {
            "Axial Strain (%)_":              "AxialStrain_Percent",
            "Volumetric Strain (%)_":         "VolumetricStrain_Percent",
            "Major Principal Stress (kPa)_":  "MajorPrincipalEffectiveStress_kPa",
            "Minor Principal Stress (kPa)_":  "MinorPrincipalEffectiveStress_kPa",
            "Pore Pressure (kPa)_":           "ExcessPorePressure_kPa",
            "Void Ratio_":                     "VoidRatio"
        }
        df_sheet.rename(columns=column_mapping, inplace=True)

        # Convertir a float los campos DoubleType para evitar errores con ints
        double_fields = [
            "AxialStrain_Percent",
            "VolumetricStrain_Percent",
            "MajorPrincipalEffectiveStress_kPa",
            "MinorPrincipalEffectiveStress_kPa",
            "ExcessPorePressure_kPa",
            "VoidRatio"
        ]
        for fld in double_fields:
            if fld in df_sheet.columns:
                df_sheet[fld] = df_sheet[fld].apply(lambda x: float(x) if pd.notnull(x) else None)

        # Añadir columnas obligatorias: LabTestID, TriaxialType, uuid, createdAt
        df_sheet["LabTestID"]    = row["Test"]
        df_sheet["TriaxialType"] = row["CD/CU"]
        df_sheet["uuid"]         = [str(uuid.uuid4()) for _ in range(len(df_sheet))]
        df_sheet["createdAt"]    = pd.Timestamp.now()

        # Rellenar cualquier columna del esquema que falte
        for col in triaxial_schema.fieldNames():
            if col not in df_sheet.columns:
                df_sheet[col] = None

        # Seleccionar únicamente las columnas definidas en el esquema
        df_sheet = df_sheet[triaxial_schema.fieldNames()]

        # Convertir NaN de pandas a None para evitar errores en Spark
        df_sheet = df_sheet.where(pd.notnull(df_sheet), None)

        return spark.createDataFrame(df_sheet, schema=triaxial_schema)

    except Exception as e:
        print(f"❌ Error procesando {row['Test']}: {str(e)}")
        return None

# Rutas de archivos en Files/Triaxials
SAMPLES_PATH = "Files/Triaxials/Sampleslist using lab numbers .xlsx"
DATA_PATH    = "Files/Triaxials/SRK-169-TX-ExcelData.xlsx"


StatementMeta(, 8465ea11-cd89-4d55-a3ff-2691f60a7fde, 18, Finished, Available, Finished)

In [18]:
# ─── Celda 2 (CORREGIDA): Carga y procesamiento de datos de Sample y LabTest ───────────

# 1. Cargar lista de muestras desde el Excel de Sample list
samples_df = load_excel(SAMPLES_PATH)

# 2. Generar la tabla de samples a partir de cada CODE único

# a) Tomar sólo un registro por “Code” (cada código de muestra debe aparecer una vez)
distinct_samples = samples_df.dropDuplicates(["Code"]) \
    .select(
        F.col("Code").alias("SampleCode"),
        F.col("Sample Type"),
        F.col("Material Group"),
        F.col("Depth (m)")
    )

# b) Añadir uuid y createdAt, luego calcular depths y asignar columnas fijas
sample_table = distinct_samples \
    .withColumn("uuid", F.expr("generate_uuid_udf(SampleCode)")) \
    .withColumn("createdAt", F.current_timestamp()) \
    .withColumn("DepthInfo", calculate_depths(F.col("Depth (m)"))) \
    .select(
        F.col("uuid"),
        F.col("SampleCode").cast("string").alias("SampleID"),
        F.lit(None).cast("string").alias("LocationID"),
        F.col("Sample Type").alias("SampleType"),
        F.col("Material Group").alias("MaterialType"),
        F.lit(None).cast("string").alias("State"),
        F.col("DepthInfo.DepthFrom_m").alias("DepthFrom_m"),
        F.col("DepthInfo.DepthTo_m").alias("DepthTo_m"),
        F.col("DepthInfo.MiddleDepth_m").alias("MiddleDepth_m"),
        F.lit(None).cast("string").alias("Laboratory"),
        F.lit(None).cast("string").alias("Comment"),
        F.col("createdAt")
    )

# c) Crear un mapping CODE → sample_uuid para usar en labtest
sample_map = sample_table.select("SampleID", "uuid").withColumnRenamed("uuid", "SampleUUID")

# 3. Generar la tabla de labtests (una fila por cada “Test” único)

# a) Seleccionar sólo un registro por Test (para evitar duplicados)
distinct_labtests = samples_df.dropDuplicates(["Test"]) \
    .select(
        F.col("Code").alias("SampleCode"),
        F.col("Test").alias("LabTestID"),
        F.col("CD/CU").alias("TestType")
    )

# b) Unir con sample_map para obtener el SampleUUID y generar uuid propio
labtest_table = distinct_labtests.join(
    sample_map,
    distinct_labtests["SampleCode"] == sample_map["SampleID"],
    how="left"
).select(
    F.expr("generate_uuid_udf(LabTestID)").alias("uuid"),
    F.col("LabTestID"),
    F.col("SampleUUID").alias("SampleID"),
    F.col("TestType"),
    F.lit(None).cast("date").alias("TestDate"),
    F.lit(None).cast("date").alias("ReceivedDate"),
    F.lit(None).cast("string").alias("Comment"),
    F.current_timestamp().alias("createdAt")
)


StatementMeta(, 8465ea11-cd89-4d55-a3ff-2691f60a7fde, 19, Finished, Available, Finished)

In [19]:
# ─── Celda 3: Escritura en Lakehouse Silver y procesamiento de Triaxial ────────────────
# Propiedades Delta para optimización y CDC
delta_properties = {
    "delta.enableChangeDataFeed": "true",
    "delta.autoOptimize.autoCompact": "true"
}

# 4. Escribir tabla Sample en silver.dbo.Sample
sample_table.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .options(**delta_properties) \
    .saveAsTable("silver.dbo.Sample")

# 5. Escribir tabla LabTest en silver.dbo.LabTest
labtest_table.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .options(**delta_properties) \
    .saveAsTable("silver.dbo.LabTest")

# 6. Procesar y escribir datos triaxiales (capa Triaxial)
triaxial_dfs = []
for row in samples_df.filter(F.col("CD/CU").isin(["CU", "CD"])).collect():
    df_triax = process_triaxial_sheet(row, DATA_PATH)
    if df_triax:
        triaxial_dfs.append(df_triax)
        print(f"✅ Triaxial procesado: {row['Test']}")

if triaxial_dfs:
    triaxial_table = triaxial_dfs[0]
    for df_next in triaxial_dfs[1:]:
        triaxial_table = triaxial_table.unionByName(df_next)

    # 7. Escribir tabla Triaxial en silver.dbo.Triaxial
    triaxial_table.write.format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .options(**delta_properties) \
        .saveAsTable("silver.dbo.Triaxial")
else:
    print("⚠️ No se procesaron datos triaxiales")

# 8. Validación y reporte final
print("\n" + "="*50)
print("RESUMEN DE CARGA EN SILVER")
print("="*50)

sample_count   = spark.sql("SELECT COUNT(*) FROM silver.dbo.Sample").first()[0]
labtest_count  = spark.sql("SELECT COUNT(*) FROM silver.dbo.LabTest").first()[0]
triaxial_count = 0
try:
    triaxial_count = spark.sql("SELECT COUNT(*) FROM silver.dbo.Triaxial").first()[0]
except:
    pass

print(f"Muestras (Sample):                {sample_count} registros")
print(f"Ensayos (LabTest):               {labtest_count} registros")
print(f"Datos triaxiales (Triaxial):     {triaxial_count} registros")


StatementMeta(, 8465ea11-cd89-4d55-a3ff-2691f60a7fde, 20, Finished, Available, Finished)

✅ Triaxial procesado: 3095-T1-CU
✅ Triaxial procesado: 3095-T2-CU
✅ Triaxial procesado: 3095-T3-CU
✅ Triaxial procesado: 3095-T1-CD
✅ Triaxial procesado: 3095-T2-CD
✅ Triaxial procesado: 3095-T3-CD
✅ Triaxial procesado: 3096-T1-CU
✅ Triaxial procesado: 3096-T2-CU
✅ Triaxial procesado: 3096-T3-CU
✅ Triaxial procesado: 3096-T1-CD
✅ Triaxial procesado: 3096-T2-CD
✅ Triaxial procesado: 3096-T3-CD
✅ Triaxial procesado: 3099-T1-CU
✅ Triaxial procesado: 3099-T2-CU
✅ Triaxial procesado: 3099-T3-CU
✅ Triaxial procesado: 3099-T1-CD
✅ Triaxial procesado: 3099-T2-CD
✅ Triaxial procesado: 3099-T3-CD
✅ Triaxial procesado: 3112-T1-CU
✅ Triaxial procesado: 3112-T2-CU
✅ Triaxial procesado: 3112-T3-CU
✅ Triaxial procesado: 3113-T1-CU
✅ Triaxial procesado: 3113-T3-CU
✅ Triaxial procesado: 3114-T1-CU
✅ Triaxial procesado: 3114-T2-CU
✅ Triaxial procesado: 3114-T3-CU
✅ Triaxial procesado: 3109-T1-CU
✅ Triaxial procesado: 3109-T2-CU
✅ Triaxial procesado: 3109-T3-CU
✅ Triaxial procesado: 3110-T1-CU
✅ Triaxial

In [27]:
%%sql
SELECT * FROM atterberg

StatementMeta(, 8465ea11-cd89-4d55-a3ff-2691f60a7fde, 28, Finished, Available, Finished)

<Spark SQL result set with 21 rows and 7 fields>